In [120]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns 

In [121]:
path_to_file ='C:/Users/etusk/Desktop/crime_file.csv' 
df = pd.read_csv(path_to_file)
df.head()

,Table 8,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
0,NEW YORK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan
1,Offenses Known to Law Enforcement,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan
2,"by City, 2013",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan
3,City,Population,Violent\ncrime,Murder and\nnonnegligent\nmanslaughter,Rape\n(revised\ndefinition)1,Rape\n(legacy\ndefinition)2,Robbery,Aggravated\nassault,Property\ncrime,Burglary,Larceny-\ntheft,Motor\nvehicle\ntheft,Arson3,nan
4,Adams Village,"1,861",0,0,NaN,0,0,0,12,2,10,0,0,nan


In [122]:
df = df.drop([0, 1, 2,]) #dropping empty rows due to titles 

In [123]:
df.columns = df.iloc[0] #aligning the correct headers 

In [124]:
df=df.drop([3]) #removed repeater header rows 
df.columns = ['city', 'population', 'violent_crime', 'murder','rape1', 'rape2', 'robbery', 'aggravated_assault', 'property_crime','burglary','theft','motor_theft','arson','none']
df = df.drop(columns=['rape1', 'none'])
df = df.drop([352, 353, 354])
# first remove the commas 
df['robbery'] = df['robbery'].str.replace(',', '')
df['population'] = df['population'].str.replace(',', '')
df['property_crime'] = df['property_crime'].str.replace(',', '')
df['theft'] = df['theft'].str.replace(',', '')
df['burglary']=df['burglary'].str.replace(',','')
#df.isin(['3,458']).any() #fixing errors - finds a value in a DF

In [125]:
print('Total number of cities:', len(df['city'].unique()))

Total number of cities: 348


In [127]:
df.dtypes

city                  object
population            object
violent_crime         object
murder                object
rape2                 object
robbery               object
aggravated_assault    object
property_crime        object
burglary              object
theft                 object
motor_theft           object
arson                 object
dtype: object

In [128]:
df.describe()

,city,population,violent_crime,murder,rape2,robbery,aggravated_assault,property_crime,burglary,theft,motor_theft,arson
count,348,348,348,348,348,348,348,348,348,348,348,187
unique,348,347,82,14,26,51,69,239,124,232,51,13
top,Menands Village,9517,0,0,0,0,0,0,0,0,0,0
freq,1,2,51,299,204,139,66,6,21,7,93,133


In [129]:
def check_nans(df):
    for column in df.columns.unique():
        print(column,":",df[column].isnull().sum()/df.shape[0]*100 ,"% is Nan")

check_nans(df)

city : 0.0 % is Nan
population : 0.0 % is Nan
violent_crime : 0.0 % is Nan
murder : 0.0 % is Nan
rape2 : 0.0 % is Nan
robbery : 0.0 % is Nan
aggravated_assault : 0.0 % is Nan
property_crime : 0.0 % is Nan
burglary : 0.0 % is Nan
theft : 0.0 % is Nan
motor_theft : 0.0 % is Nan
arson : 46.264367816091955 % is Nan


In [130]:
# Sort data by population - Look for outliers 
df.sort_values(by=['population'], ascending =False).head()
df= df[df['city'] != 'New York'] #dropping NY to get better eresults as population is an outlier

In [131]:
df.city = df.city.str.lstrip() #Removes whitespace 
df.city = df.city.str.rstrip()

In [132]:
# change robbery, and population to int.
df[['robbery', 'population', 'murder']] = df[['robbery', 'population', 'murder']].apply(pd.to_numeric)

In [133]:
df.head() #Data is useable
# Generate new features based on the instruction of the assignment - Why is this necessary
#df2['Population^2']=df2['Population']**2
#df2['Murder_Bi']=np.where(df2['Murder and nonnegligent manslaughter']>0, 1, 0)
#df2['Robbery_Bi']=np.where(df2['Robbery']>0, 1, 0)
#df2.head()

,city,population,violent_crime,murder,rape2,robbery,aggravated_assault,property_crime,burglary,theft,motor_theft,arson
4,Adams Village,1861,0,0,0,0,0,12,2,10,0,0
5,Addison Town and Village,2577,3,0,0,0,3,24,3,20,1,0
6,Akron Village,2846,3,0,0,0,3,16,1,15,0,0
7,Albany,97956,791,8,30,227,526,4090,705,3243,142,NaN
8,Albion Village,6388,23,0,3,4,16,223,53,165,5,NaN


In [134]:
import math
import warnings

from IPython.display import display
from sklearn import linear_model

# Display preferences.
%matplotlib inline
pd.options.display.float_format = '{:.3f}'.format

# Suppress annoying harmless error.
warnings.filterwarnings(
    action="ignore",
    module="scipy",
    message="^internal gelsd"
)

In [135]:
# Instantiate and fit our model. - Seems to Overfit - What do I do now? 
regr = linear_model.LinearRegression()
Y = df['property_crime'].values.reshape(-1, 1)
X = df[['population','burglary', 'theft']]
regr.fit(X, Y)

# Inspect the results.
print('\nCoefficients: \n', regr.coef_)
print('\nIntercept: \n', regr.intercept_)
print('\nR-squared:')
print(regr.score(X, Y))


Coefficients: 
 [[5.91643995e-04 1.29851561e+00 9.61510989e-01]]

Intercept: 
 [-4.89241067]

R-squared:
0.9998470421748513


In [141]:
#Extract predicted values. - Why am I getting error? This is trying to check for multivariable normality and homoscdancity 
predicted = regr.predict(X).ravel()
actual = Y.ravel()

#Calculate the error, also called the residual.
residual = actual - predicted

# Error histogram
plt.hist(residual)
plt.title('Residual counts')
plt.xlabel('Residual')
plt.ylabel('Count')
plt.show()

# Error scatter plot
plt.scatter(predicted, residual)
plt.xlabel('Predicted')
plt.ylabel('Residual')
plt.axhline(y=0)
plt.title('Residual vs. Predicted')
plt.show()

TypeError: unsupported operand type(s) for -: 'str' and 'float'

AttributeError: 'DataFrame' object has no attribute 'dtype'